In [38]:
%matplotlib inline 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pprint
pp = pprint.PrettyPrinter(indent=4)

import GPy

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [39]:
movie = pd.read_csv(r'data/movie_normal.csv')
rating = pd.read_csv(r'data/rating.csv')
# movie.head()
# rating.head()

In [40]:
#user_id to train on
id=4169

In [41]:
def get_train_for_user(id=1):
    user_ratings = rating[rating['user_id']==id]
    return pd.merge(user_ratings, movie, on='movie_id')

def shuffle(df, f=.8):
    df = df.reindex(np.random.permutation(df.index))
    df.reset_index()
    return df

In [42]:
# get all data for specified user
data = get_train_for_user(id=id).drop(['user_id', 'movie_id', 'title'], axis=1)

In [43]:
# split into features and target
X_data = data.ix[:,1:]
Y_data = data.ix[:,0]
X_data.shape, Y_data.shape

((2264, 25), (2264,))

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=0)

In [45]:
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(X_train)

In [46]:
y_train.values.shape

(1584,)

In [47]:
X_train1 = X_train_minmax
y_train1 = y_train.values
y_train1 = y_train1.reshape((1584,1))
kernel = GPy.kern.RBF(input_dim=len(X_train_minmax[0]), variance=1., lengthscale=1.)

In [48]:
m = GPy.models.GPRegression(X_train1,y_train1,kernel)
m.optimize()

In [49]:
X_test1 = mm_scaler.transform(X_test)

In [50]:
# predicted values and another output.
# I guess it's the variance but I can't find docs on it
Yhat, conf = m.predict(X_test1)

In [51]:
# probably there's a better way to do this...
act=pd.DataFrame(y_test).reset_index(drop=True)
prd=pd.DataFrame(Yhat)
conf=pd.DataFrame(conf)
diff=pd.concat([act, prd, conf], axis=1,join_axes=[act.index])

In [52]:
diff.columns=['actual', 'predicted', 'conf']

In [53]:
diff['residual'] = diff.apply(lambda r: r['predicted']-r['actual'], axis=1)
diff['abs_residual'] = diff.apply(lambda r: abs(r['predicted']-r['actual']), axis=1)

In [54]:
print(diff.sort_values(by='residual').head())
print(diff.sort_values(by='residual').tail())

     actual  predicted      conf  residual  abs_residual
163       5   2.346145  0.694532 -2.653855      2.653855
563       5   2.451226  0.719291 -2.548774      2.548774
203       5   2.756723  0.708371 -2.243277      2.243277
133       5   2.777833  0.714102 -2.222167      2.222167
371       5   2.854599  0.691869 -2.145401      2.145401
     actual  predicted      conf  residual  abs_residual
518       1   3.247893  0.691596  2.247893      2.247893
119       1   3.312579  0.723393  2.312579      2.312579
575       1   3.366000  0.690296  2.366000      2.366000
501       1   3.928033  0.709684  2.928033      2.928033
389       1   4.482198  0.700052  3.482198      3.482198


In [55]:
print(diff.sort_values(by='abs_residual').head())
print(diff.sort_values(by='abs_residual').tail())

     actual  predicted      conf  residual  abs_residual
462       5   5.003377  0.723787  0.003377      0.003377
169       3   2.995594  0.697093 -0.004406      0.004406
82        3   2.994206  0.679701 -0.005794      0.005794
650       4   4.008168  0.695069  0.008168      0.008168
99        3   2.986495  0.751248 -0.013505      0.013505
     actual  predicted      conf  residual  abs_residual
575       1   3.366000  0.690296  2.366000      2.366000
563       5   2.451226  0.719291 -2.548774      2.548774
163       5   2.346145  0.694532 -2.653855      2.653855
501       1   3.928033  0.709684  2.928033      2.928033
389       1   4.482198  0.700052  3.482198      3.482198


In [33]:
diff.describe()